In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
from vis_L23_util import get_data_dir

from util_preprocessing import filter_synapses

data_folder = get_data_dir() / "VIS-L23"
soma_file = data_folder / "meta" / "soma.csv"

In [2]:
somas = pd.read_csv(soma_file)
#print(somas)

celltype_map = {} # neuron id -> celltype
for _, row in somas.iterrows():
    celltype_map[row["neuron_id_mapped"].astype(int)] = row["celltype"].astype(int)

In [3]:
ids_exc = sorted(somas[somas["axon_length"] >= 100]["neuron_id_mapped"].to_numpy(int))
ids_inh = sorted(somas[somas["celltype"] != 1]["neuron_id_mapped"].to_numpy(int))

ids_selected = ids_exc + ids_inh

np.savetxt(data_folder / "meta" / "ids_selected_EXC", ids_exc, fmt="%d")
np.savetxt(data_folder / "meta" / "ids_selected_INH", ids_inh, fmt="%d")
np.savetxt(data_folder / "meta" / "ids_selected", ids_selected, fmt="%d")

In [4]:
filter_synapses(data_folder/"synapses_classified_neurons.csv", data_folder/"synapses_selected_neurons.csv", ids_selected)

In [5]:
synapses_filtered = pd.read_csv(data_folder/"synapses_selected_neurons.csv")[["pre_id", "post_id", "pre_celltype"]].to_numpy(int)

ids = []
celltypes = []
num_outgoing = []
num_incoming = []

for neuron_id in ids_selected:
    ids.append(neuron_id)
    celltypes.append(celltype_map[neuron_id])
    mask_pre = synapses_filtered[:,0] == neuron_id
    mask_post = synapses_filtered[:,1] == neuron_id    
    num_outgoing.append(np.count_nonzero(mask_pre))
    num_incoming.append(np.count_nonzero(mask_post))

data = {
    "neuron_id" : ids,
    "celltype" : celltypes,
    "outgoing_synapses" : num_outgoing,
    "incoming_synapses" : num_incoming
}

selected_neuron_properties = pd.DataFrame(data)
selected_neuron_properties.to_csv(data_folder/"meta"/"selected_neuron_properties.csv", index=False)

In [6]:
df = selected_neuron_properties

df_filtered = df[((df["outgoing_synapses"] >= 20) & (df["celltype"] == 1)) | ((df["outgoing_synapses"] >= 4) & (df["celltype"] != 1))]

print(df_filtered.shape)
print(np.unique(df_filtered["celltype"].to_numpy(int), return_counts=True))

ids_single_cell = sorted(df_filtered["neuron_id"].to_numpy(int))
np.savetxt(data_folder / "meta" / "ids_single_cell_analysis", ids_single_cell, fmt="%d")

(47, 4)
(array([ 1, 20, 21, 22, 23, 24, 25]), array([33,  3,  3,  4,  2,  1,  1]))
